# Transcripción Automática de Videos con Whisper

**Propósito:** Extraer el audio de videos y transcribirlo a texto de forma simple y rápida.

**Modelos:** FFmpeg + OpenAI Whisper

---

## ¿Qué vamos a hacer?

En este cuaderno vas a aprender a:
1. Extraer el audio de archivos de video
2. Transcribir ese audio a texto usando Whisper

**Casos de uso:**
- Generar subtítulos para videos
- Transcribir conferencias y presentaciones grabadas
- Documentar entrevistas en video
- Crear textos a partir de tutoriales en video
- Hacer videos accesibles con texto

**Requisitos:**
- Google Colab con GPU habilitada (Runtime > Change runtime type > GPU)
- Archivo de video en formato común (MP4, AVI, MOV, MKV, etc.)

## Paso 1: Instalación de Dependencias

Instalamos las librerías necesarias para trabajar con videos y Whisper.

In [ ]:
# Instalar dependencias
!pip install -U -q transformers torch torchvision accelerate gradio moviepy

print("✅ Dependencias instaladas correctamente")

## Paso 2: Importar Librerías y Configurar

Importamos las librerías necesarias.

In [ ]:
import torch
from transformers import pipeline
import gradio as gr
from moviepy.editor import VideoFileClip
import os
import warnings

warnings.filterwarnings('ignore')

print("📦 Librerías importadas correctamente")

### Selección del Modelo de Whisper

| Modelo | Tamaño | Velocidad | Precisión | Uso recomendado |
|--------|--------|-----------|-----------|------------------|
| tiny | ~39 MB | Muy rápido | Básica | Pruebas rápidas |
| base | ~74 MB | Rápido | Buena | Videos cortos |
| small | ~244 MB | Moderado | Muy buena | **Recomendado** |
| medium | ~769 MB | Lento | Excelente | Videos largos |
| large-v3 | ~1550 MB | Muy lento | Máxima | Máxima calidad |

In [ ]:
# Seleccionar modelo
MODELO_WHISPER = "openai/whisper-small"  # Cambia a tiny, base, medium o large-v3

print(f"🤖 Modelo seleccionado: {MODELO_WHISPER}")

## Paso 3: Cargar el Modelo de Whisper

In [ ]:
# Detectar dispositivo
device = 0 if torch.cuda.is_available() else -1
torch_dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32

print(f"💻 Dispositivo: {'GPU ✅' if device == 0 else 'CPU ⚠️'}")
if device == -1:
    print("⚠️ No hay GPU disponible. El proceso será más lento.")
    print("   Para usar GPU: Runtime > Change runtime type > GPU")

# Cargar modelo
print(f"\n🔄 Cargando modelo {MODELO_WHISPER}...")
print("   (Esto puede tardar unos minutos la primera vez)\n")

transcriptor = pipeline(
    "automatic-speech-recognition",
    model=MODELO_WHISPER,
    device=device,
    torch_dtype=torch_dtype,
    return_timestamps=True,
    chunk_length_s=30,
    stride_length_s=5
)

print("✅ Modelo cargado exitosamente\n")
print("🎯 Listo para procesar videos")

## Paso 4: Función para Extraer Audio del Video

Creamos una función que extrae el audio de un archivo de video.

In [ ]:
def extraer_audio_de_video(ruta_video):
    """
    Extrae el audio de un archivo de video y lo guarda como WAV.

    Args:
        ruta_video: Ruta al archivo de video

    Returns:
        str: Ruta al archivo de audio extraído
    """
    try:
        print("🎬 Extrayendo audio del video...")

        # Cargar video
        video = VideoFileClip(ruta_video)

        # Generar nombre para el archivo de audio
        nombre_base = os.path.splitext(ruta_video)[0]
        ruta_audio = f"{nombre_base}_audio.wav"

        # Extraer y guardar audio
        video.audio.write_audiofile(ruta_audio, verbose=False, logger=None)

        # Cerrar video para liberar recursos
        video.close()

        duracion = video.duration
        print(f"✅ Audio extraído exitosamente")
        print(f"   Duración: {int(duracion//60)}:{int(duracion%60):02d} minutos")
        print(f"   Archivo: {ruta_audio}")

        return ruta_audio

    except Exception as e:
        error = f"❌ Error al extraer audio: {str(e)}"
        print(error)
        return None

print("✅ Función de extracción de audio definida")

## Paso 5: Función de Transcripción de Video

Función principal que combina la extracción de audio y la transcripción.

In [ ]:
def transcribir_video(archivo_video):
    """
    Transcribe un archivo de video a texto.

    Args:
        archivo_video: Ruta al archivo de video

    Returns:
        str: Transcripción del video
    """
    if not archivo_video:
        return "❌ No se cargó ningún archivo de video"

    try:
        # Paso 1: Extraer audio
        ruta_audio = extraer_audio_de_video(archivo_video)

        if not ruta_audio:
            return "❌ No se pudo extraer el audio del video"

        # Paso 2: Transcribir audio
        print("\n🎯 Iniciando transcripción...")
        resultado = transcriptor(ruta_audio)

        # Extraer texto
        texto = ""
        if isinstance(resultado, dict) and "text" in resultado:
            texto = resultado["text"].strip()
        elif isinstance(resultado, str):
            texto = resultado.strip()

        if not texto:
            return "❌ No se pudo extraer texto del audio"

        print(f"✅ Transcripción completada: {len(texto)} caracteres")

        # Limpiar archivo de audio temporal
        try:
            os.remove(ruta_audio)
            print("🧹 Archivo de audio temporal eliminado")
        except:
            pass

        # Formatear salida
        salida = f"🎬 TRANSCRIPCIÓN DE VIDEO\n"
        salida += f"{'='*60}\n\n"
        salida += texto
        salida += f"\n\n{'='*60}\n"
        salida += f"📊 Total de caracteres: {len(texto)}\n"
        salida += f"📊 Palabras aproximadas: {len(texto.split())}\n"
        salida += f"🤖 Modelo utilizado: {MODELO_WHISPER}\n"
        salida += f"📁 Archivo: {os.path.basename(archivo_video)}"

        return salida

    except Exception as e:
        error = f"❌ Error durante el procesamiento:\n{str(e)}"
        print(error)
        return error

print("✅ Función de transcripción de video definida")

## Paso 6: Interfaz Interactiva con Gradio

In [ ]:
# Crear interfaz
interfaz = gr.Interface(
    fn=transcribir_video,
    inputs=gr.Video(
        label="🎬 Subir Archivo de Video",
        sources=["upload"]
    ),
    outputs=gr.Textbox(
        label="📝 Transcripción",
        lines=15,
        show_copy_button=True,
        placeholder="La transcripción del video aparecerá aquí..."
    ),
    title="🎬 Transcriptor de Videos con Whisper",
    description=f"""
    ### 🚀 Transcripción automática de videos a texto

    **Modelo:** {MODELO_WHISPER}
    **Dispositivo:** {'GPU' if device == 0 else 'CPU'}

    #### 📋 Instrucciones:
    1. Sube tu archivo de video (MP4, AVI, MOV, MKV, etc.)
    2. Espera a que se extraiga el audio y se transcriba
    3. Copia o descarga la transcripción

    #### 🔄 Proceso:
    1. Extracción del audio del video
    2. Transcripción del audio con Whisper
    3. Limpieza de archivos temporales

    #### ✨ Características:
    - Soporta múltiples formatos de video
    - Detecta automáticamente el idioma
    - Alta precisión de transcripción
    - Procesa videos de cualquier duración

    #### ⚠️ Importante:
    - Videos largos pueden tardar varios minutos
    - El audio se extrae en formato WAV automáticamente
    - Se recomienda usar GPU para videos largos
    """,
    examples=[],
    cache_examples=False,
    allow_flagging="never",
    theme=gr.themes.Soft()
)

print("✅ Interfaz creada")

## Paso 7: Lanzar la Aplicación

In [ ]:
# Lanzar aplicación
print("🚀 Lanzando aplicación...\n")

interfaz.launch(
    share=True,
    debug=True,
    show_error=True
)

print("""
✅ Aplicación en ejecución

💡 Consejos:
- Videos con audio claro dan mejores resultados
- El procesamiento puede tardar dependiendo de la duración del video
- Podes copiar la transcripción con el botón de copiar
- El audio se extrae automáticamente y se elimina después

⏱️ Tiempos estimados:
- Video de 1 minuto: ~30 segundos
- Video de 5 minutos: ~2-3 minutos
- Video de 10 minutos: ~5-7 minutos

⚠️ Importante:
- Esta aplicación se ejecuta mientras esta celda esté corriendo
- Para detenerla, presiona el botón de stop en esta celda
""")

---

## Opción Alternativa: Transcribir Directamente desde Código

In [ ]:
# Subir archivo manualmente
from google.colab import files

print("🎬 Selecciona tu archivo de video:")
archivo_subido = files.upload()

# Obtener nombre del archivo
nombre_archivo = list(archivo_subido.keys())[0]
print(f"\n✅ Archivo cargado: {nombre_archivo}")

In [ ]:
# Transcribir el video
resultado = transcribir_video(nombre_archivo)
print(resultado)

In [ ]:
# Opcional: Guardar transcripción en archivo
nombre_salida = nombre_archivo.rsplit('.', 1)[0] + '_transcripcion.txt'

with open(nombre_salida, 'w', encoding='utf-8') as f:
    f.write(resultado)

print(f"💾 Transcripción guardada en: {nombre_salida}")

# Descargar el archivo
files.download(nombre_salida)

---

## Bonus: Generar Subtítulos con Timestamps

Si necesitas timestamps para generar subtítulos (formato SRT), usa esta función:

In [ ]:
def transcribir_con_timestamps(archivo_video):
    """
    Transcribe un video y retorna el texto con timestamps (útil para subtítulos).
    """
    if not archivo_video:
        return "❌ No se cargó ningún archivo de video"

    try:
        # Extraer audio
        ruta_audio = extraer_audio_de_video(archivo_video)
        if not ruta_audio:
            return "❌ No se pudo extraer el audio"

        # Transcribir con timestamps
        print("\n🎯 Transcribiendo con timestamps...")
        resultado = transcriptor(ruta_audio, return_timestamps="word")

        # Formatear salida con timestamps
        salida = "🎬 TRANSCRIPCIÓN CON TIMESTAMPS\n"
        salida += "="*60 + "\n\n"

        if "chunks" in resultado:
            for i, chunk in enumerate(resultado["chunks"], 1):
                timestamp = chunk.get("timestamp", [0, 0])
                inicio = timestamp[0] if timestamp[0] else 0
                fin = timestamp[1] if timestamp[1] else 0
                texto = chunk.get("text", "")

                salida += f"[{inicio:.2f}s - {fin:.2f}s] {texto}\n"

        # Limpiar archivo temporal
        try:
            os.remove(ruta_audio)
        except:
            pass

        return salida

    except Exception as e:
        return f"❌ Error: {str(e)}"

print("✅ Función de transcripción con timestamps definida")

In [ ]:
# Usar la función con timestamps
# resultado_timestamps = transcribir_con_timestamps(nombre_archivo)
# print(resultado_timestamps)

---

## Ejercicios Prácticos

### Ejercicio 1: Videos de Diferentes Formatos
1. Prueba con videos en diferentes formatos (MP4, AVI, MOV, MKV)
2. ¿Todos los formatos funcionan correctamente?
3. ¿Hay diferencias en el tiempo de procesamiento?

### Ejercicio 2: Videos con Múltiples Hablantes
1. Transcribe un video con varias personas hablando
2. ¿Whisper puede distinguir entre hablantes?
3. ¿Cómo afecta esto a la calidad de la transcripción?

### Ejercicio 3: Generar Subtítulos SRT
Modifica la función con timestamps para generar archivos .srt:
```
1
00:00:00,000 --> 00:00:03,500
Primera línea de subtítulo

2
00:00:03,500 --> 00:00:07,000
Segunda línea de subtítulo
```

### Ejercicio 4: Comparar Calidad de Video
1. Toma el mismo video en diferentes calidades
2. Transcribe cada versión
3. ¿La calidad del video afecta la transcripción?

---

## Solución de Problemas

### Error al Extraer Audio
- Verifica que el video tenga audio
- Prueba con otro formato de video
- Asegurate que el archivo no esté corrupto

### Out of Memory
- Usa un modelo más pequeño (tiny o base)
- Divide el video en segmentos más cortos
- Reinicia el runtime

### Transcripción Imprecisa
- Usa un modelo más grande (medium o large-v3)
- Verifica la calidad del audio en el video
- Asegurate que no haya mucho ruido de fondo

### Proceso Muy Lento
- Verifica que estés usando GPU
- Usa un modelo más pequeño
- Reduce la duración del video

---

## Recursos Adicionales

- [Whisper GitHub](https://github.com/openai/whisper)
- [MoviePy Documentation](https://zulko.github.io/moviepy/)
- [FFmpeg Documentation](https://ffmpeg.org/documentation.html)
- [Formato SRT para Subtítulos](https://en.wikipedia.org/wiki/SubRip)

---

**Creado para el curso de Procesamiento del Lenguaje Natural**  
**Última actualización:** Noviembre 2025